## 第六章 栈、队列和双端队列

### 6.1 栈

栈（stack）：一系列对象组成的collection，对象的插入和删除操作遵循后进先出（LIFO）的原则。

在栈中插入、删除、访问的对象只能是栈顶。（自动售卖机）

栈的例子：历史网址，在后退时按照的顺序就是后进先出，网址用栈来存储；撤销功能，将每一步操作存储在栈中，撤销时也是按照后进先出的顺序。

#### 6.1.1 栈的抽象数据类型

栈是一种支持以下两种操作的抽象数据类型（ADT）：

1. 将一个元素添加到栈顶。（push）
2. 移除并返回栈顶的元素。如果栈为空，该操作报错或出错。（pop）

除操作方法外还有访问方法：

1. 返回栈顶元素但不移除，栈为空将报错。（top）
2. 返回栈是否为空。（is_empty）
3. 返回栈中元素的个数。（len）

#### 6.1.2 简单的基于数组的栈实现

虽然数组或者list可以实现栈的功能，但是同时也包含了许多栈不能实现的功能，不能与栈混为一谈。

适配器模式（The Adapter Pattern）：适配器设计模式用于将一个类改编为另一个类，两个类可能在有些功能上相同，但需要不同的命名或者接口，可以通过建立一个新类，新类中用一个旧类的实例来初始化以及实现类的方法。

通过适配器模式用Python的list类来构造一个stack类。

通过list类构造stack类（底层是一个array），在stack类中，只允许特定操作和访问的存在，不允许其他非法操作，这能够起到一些限制作用，可能也是使用stack而不是array来存储一些元素的优点。

对于list而言，pop和top（假设已经在stack类中实现）在list为空时会触发IndexError，由于stack没有index，所以需要重新定义一个新的异常类。

In [1]:
class Empty(Exception):
    pass

class ArrayStack:
    
    def __init__(self):
        self._data = []
    
    def __len__(self):
        return len(self._data)
    
    def is_empty(self):
        return self._data == []
    
    def push(self, e):
        self._data.append(e)
    
    def top(self):
        if self._data:
            return self._data[-1]
        else:
            raise Empty('Stack is empty!')
    
    def pop(self):
        if self._data:
            return self._data.pop()
        else:
            raise Empty('Stack is empty!')

In [2]:
mystack = ArrayStack()
mystack.push(3)
mystack.push(4)
print(mystack.pop())
print(mystack.top())
print(len(mystack))
print(mystack.is_empty())
print(mystack.pop())
print(mystack.is_empty())
print(mystack.pop())

4
3
1
False
3
True


Empty: Stack is empty!

栈的所有操作和访问都是常量时间，上面实现的ArrayStack的pop和push操作要进行摊销分析。

#### 6.1.3 使用栈实现数据的逆置

以文件为例，逆序打印每一行：

In [2]:
def reverse_file(filename):
    S = ArrayStack()
    original = open(filename)
    for line in original:
        S.push(line.rstrip('\n'))
    original.close()
    
    output = open(filename, 'w')
    while not S.is_empty():
        output.write(S.pop() + '\n')
    output.close()

一个序列按照一个顺序压入栈中，再依次弹出将得到反序的序列。

#### 6.1.4 括号和HTML标记匹配

在算术表达式中，括号要成对出现，即不能多出一边括号，同时，一对括号中的括号也要成对出现，不能一个括号在一对括号中，另一个在这对括号之外。要判断算术表达式是否正确，可以使用栈，在遇到左括号时压入栈，然后遇到右括号时与栈顶的括号进行判断，如果栈为空，报错；如果不匹配，报错；遍历完算术表达式的每个字符后，如果过程中未报错，且栈为空，则算术表达式正确。

In [3]:
def is_matched(expr):
    lefty = '({['
    righty = ')}]'
    S = ArrayStack()
    for c in expr:
        if c in lefty:
            S.push(c)
        elif c in righty:
            if S.is_empty():
                return False
            if righty.index(c) != lefty.index(S.pop()):
                return False
    return S.is_empty()

若算术表达式长度为n，则上述算法时间复杂度为O(n)。

类似的思想，可以用于HTML等标记语言中检查标签是否成对出现。

In [8]:
def is_matched_html(raw):
    S = ArrayStack()
    j = raw.find('<')
    while j != -1:
        k = raw.find('>', j+1)
        if k == -1:
            return False
        tag = raw[(j+1):k]
        if not tag.startswith('/'):
            S.push(tag)
        else:
            if S.is_empty():
                return False
            if tag[1:] != S.pop():
                return False
        j = raw.find('<', k+1)
    return S.is_empty()

### 6.2 队列

队列（queues）：一系列对象组成的collection，对象的插入和删除操作遵循先入先出（FIFO）的原则。

插入元素只能插入到队尾，删除元素只能删除队首的元素。（通常将允许插入元素的一端称为队尾，允许删除元素的一端称为队首）

跟排队有关的都可以用队列存储，如打印机、网址请求等等。

队列的抽象数据类型（ADT）是一系列对象的collection，定义的基本操作是：

1. Q.enqueue(e)：向队列Q的队尾添加一个元素（入队）。
2. Q.dequeue(e)：向队列Q中移除并返回第一个元素（离队），队列为空则触发错误。

还定义了一些访问方法：

1. Q.first()：返回队列的第一个元素但不移除，如果队列为空则触发错误。
2. Q.is_empty()：队列是否为空。
3. len(Q)：返回队列Q中元素的数量。

#### 6.2.2 基于数组的队列实现

将Python的list类作为适配器类，编写queues类。

由于数组移除第一个元素效率很低，所以不能很简单地跟栈一样改编，而要进行更大程度上优化。

可以不将队列的元素原封不动的复制到list，而是用一个index来表示list中属于队列元素的部分，但是这样可能导致一边出队一边入队很多次后，list的长度远远长于queues。也就是所有曾经入队过的元素都会在list中，那么长时间积累下来将十分耗费内存。

一种比较好的解决办法是，创建一个长度为N的数组，使队首从接近list末尾的地方开始，让队列的元素从list末尾向list开头延伸，在入队操作时，只需将list开头部分后的元素更新，离队时只需更新索引，假设队首开始的索引是f，那么更新索引的办法是(f+1)%N，这样f=N-1时，能将索引更新到0，在不断离队和入队的操作过程中，队列的队首和队尾在list中的位置是循环移动的，在这个过程中只需要保证queues的总长度不超过N即可。

队列的代码实现如下（基于循环lsit）：

In [5]:
class Empty(Exception):
    pass

class ArrayQueue:
    
    DEFAULT_CAPACITY = 10
    
    def __init__(self):
        self._data = [None] * ArrayQueue.DEFAULT_CAPACITY
        self._size = 0
        self._front = 0
    
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0
    
    def first(self):
        if self.is_empty():
            raise Empty('Queue is empty')
        return self._data[self._front]
    
    def dequeue(self):
        if self.is_empty():
            raise Empty('Queue is empty')
        answer = self._data[self._front]
        self._data[self._front] = None
        self._front = (self._front + 1) % len(self._data)
        self._size -= 1
        return answer
    
    def enqueue(self):
        if self._size == len(self._data):
            self._resize(2 * len(self.data))
        avail = (self._font + self._size) % len(self._data)
        self._data[avail] = e
        self._size += 1
    
    def resize(self, cap):
        old = self._data
        self._data = [None] * cap
        walk = self._front
        for k in range(self.size):
            self._data[k] = old[walk]
            walk = (1 + walk) % len(old)
        self._front = 0

这种实现方式下，底层数组的长度虽然不必为进入过队列的元素个数，但是由于没有数组的缩减操作，将导致数组的长度与出现过的最长的队列长度相近，仍然会造成浪费。因此新增一个策略，当队列的长度降到数组长度的1/4时，缩减数组为原来的1/2。

基于数组的队列实现的各种操作的时间复杂度均为O(1)，由于增加和减少元素涉及动态数组，因此会涉及到摊销分析的方法。从这里大体可以看出queues与array的差别了，如果是array直接来实现离队的操作，需要O(n)，而以数组为基础，使用循环数组实现的queues，却能达到O(1)的时间复杂度，这也是为什么在特定的情境下要用特定的数据结构，因为速度快，花费的时间少。

### 6.3 双端队列

双端队列（double-ended queue、deque）：支持在队列的头部和尾部都进行插入和删除操作。

双端队列的抽象数据类型（ADT，从更具体的层面定义了双端队列的操作）：

1. add_first(e): 向双端队列的前面添加一个元素e。
2. add_last(e): 在双端队列的后面添加一个元素e。
3. delete_first(): 从双端队列中移除并返回第一个元素。若双端队列为空，则触发错误。
4. delete_last(): 从双端队列中移除并返回最后一个元素。若双端队列为空，则触发错误。

还有一些访问操作：

1. first(): 返回（但不移除）双端队列的第一个元素。若双端队列为空，则触发错误。
2. last(): 返回（但不移除）双端队列的最后一个元素。若双端队列为空，则触发错误。
3. is_empty(): 是否为空。
4. len(): 返回双端队列中元素的个数。

双端队列的数组实现方法：与队列一样。

colllections模块实现了双端队列，并且定义了一些新的方法，如访问第j个，清除所有内容等，这些在抽象数据类型中没有定义，但是加上也无伤大雅。只要保证添加和删除元素只能在首尾即可。

### 6.4 练习

R-6.2

In [9]:
class Empty(Exception):
    pass

class ArrayStack:
    
    def __init__(self):
        self._data = []
    
    def __len__(self):
        return len(self._data)
    
    def is_empty(self):
        return self._data == []
    
    def push(self, e):
        self._data.append(e)
    
    def top(self):
        if self._data:
            return self._data[-1]
        else:
            raise Empty('Stack is empty!')
    
    def pop(self):
        if self._data:
            return self._data.pop()
        else:
            raise Empty('Stack is empty!')
    
    def __str__(self):
        return str(self._data)
     
def transfer(S, T):
    for i in range(len(S)):
        T.push(S.pop())
    return T

In [12]:
S = ArrayStack()
T = ArrayStack()

for i in range(10):
    S.push(i)

print(S)
transfer(S, T)
print(T)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


R-6.4

In [15]:
class Empty(Exception):
    pass

class ArrayStack:
    
    def __init__(self):
        self._data = []
    
    def __len__(self):
        return len(self._data)
    
    def is_empty(self):
        return self._data == []
    
    def push(self, e):
        self._data.append(e)
    
    def top(self):
        if self._data:
            return self._data[-1]
        else:
            raise Empty('Stack is empty!')
    
    def pop(self):
        if self._data:
            return self._data.pop()
        else:
            raise Empty('Stack is empty!')
    
    def remove_all(self):
        if len(self._data) == 1:
            return self.pop()
        else:
            self.pop()
            self.remove_all()
    
    def __str__(self):
        return str(self._data)

In [16]:
S = ArrayStack()
for i in range(10):
    S.push(i)

print(S)
S.remove_all()
print(S)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[]


R-6.5

In [19]:
def myreverse(mylist):
    mystack = ArrayStack()
    for i in range(len(mylist)):
        mystack.push(mylist.pop(0))
    for i in range(len(mystack)):
        mylist.append(mystack.pop())
    return mylist
    
mylist = [k for k in range(10)]
print(myreverse(mylist))

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


R-6.10

因为底层数组的长度要比队列长，因此从索引0开始循环取队列的数，可能导致队列中的数无法完全迁移，中间会出现一些None值。

R-6.11

In [36]:
import collections

class MyQueue:
    
    def __init__(self):
        self._queue = collections.deque()
        
    def __len__(self):
        return len(self._queue)
    
    def is_empty(self):
        return len(self) == 0
    
    def enqueue(self, value):
        self._queue.append(value)
    
    def dequeue(self):
        return self._queue.pop()
    
    def first(self):
        return self._queue[1]
    
    def __str__(self):
        return str(self._queue)

In [39]:
myqueue = MyQueue()

print(myqueue)

for i in range(10):
    myqueue.enqueue(i)

print(myqueue)
print(myqueue.first())

for i in range(10):
    myqueue.dequeue()

print(myqueue)

deque([])
deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
1
deque([])


C-6.16

In [46]:
class full(Exception):
    pass

class Empty(Exception):
    pass

class ArrayStack:
    
    def __init__(self, maxlen):
        self._data = []
        self._maxlen = maxlen
    
    def __len__(self):
        return len(self._data)
    
    def is_empty(self):
        return self._data == []
    
    def push(self, e):
        if len(self) == self._maxlen:
            raise full('Stack is too long!')
        else:
            self._data.append(e)
        
    def top(self):
        if self._data:
            return self._data[-1]
        else:
            raise Empty('Stack is empty!')
    
    def pop(self):
        if self._data:
            return self._data.pop()
        else:
            raise Empty('Stack is empty!')
    
    def remove_all(self):
        if len(self._data) == 1:
            return self.pop()
        else:
            self.pop()
            self.remove_all()
    
    def __str__(self):
        return str(self._data)

In [47]:
mystack = ArrayStack(10)

for i in range(11):
    mystack.push(i)

full: Stack is too long!

C-6.17

In [59]:
class full(Exception):
    pass

class Empty(Exception):
    pass

class ArrayStack:
    
    def __init__(self, maxlen):
        self._data = [None] * maxlen
        self._first = 0
        self._last = 0
        self._maxlen = maxlen
    
    def __len__(self):
        return (self._last - self._first)
    
    def is_empty(self):
        return len(self) == 0
    
    def push(self, value):
        if len(self) == self._maxlen:
            raise full('Stack is too long!')
        else:
            self._data[self._last] = value
            self._last += 1
        
    def top(self):
        if not self.is_empty():
            return self._data[-1]
        else:
            raise Empty('Stack is empty!')
    
    def pop(self):
        if not self.is_empty():
            value = self._data[self._last - 1]
            self._data[self._last - 1] = None
            self._last -= 1
            return value
        else:
            raise Empty('Stack is empty!')
    
    def remove_all(self):
        if len(self) == 1:
            return self.pop()
        else:
            self.pop()
            self.remove_all()
    
    def __str__(self):
        return str(self._data[self._first:self._last])

In [60]:
mystack = ArrayStack(10)

for i in range(10):
    mystack.push(i)
    
print(mystack)

for i in range(10):
    mystack.pop()

print(mystack)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[]


C-6.20

stack对range(n)中n个数实施n次push操作，在中间穿插pop操作，可以输出不同的排列顺序，这是基本思想。

C-6.26

两个栈实现队列，每次队列的enqueue操作，都将value也push到第一个栈中，每次deque操作，都将第一个栈中所有的value都弹出，压入到第二个栈中，然后进行一次pop，然后从第二个栈中弹出所有元素并压入第一个栈中。